<h1>PageRank for Anomaly Detection</h1>

The PageRank algorithm is a centrality measure that ranks the relative importance of nodes in a directed graph. It was invented by the founders of Google - Larry Page and Sergey Brin. The alogrithm was originally intended to rank webpages based on the simple idea that a page is important if many other important pages link to it.

<h2>Concept</h2>

Consider a person "surfing" random websites:
1. The person starts on a random page.
2. From the page they are currently on they can either:
   * Click on a link to an external page, with probablity $\alpha$ (usually 0.85)
   * They can <i>teleport</i> (ie type in a url) to any random page, with probablity $1-\alpha$
3. After many hours of surfing, the probability of being on each page reaches a steady state (stabilizes). These steady state probabilities are the PageRank scores.

<h2>Notation</h2>


| Symbol | Meaning |
|:--|:--|
|  $N$  | Total number of nodes |
|  $M$  | Column-stochastic adjacency matrix (each column sums to 1) |
|  $M_{ij} = 1/k_j$  | Probability of moving from node $j$ to node $i$, where $k_j$ is the out-degree of node $j$ |
|  $\alpha \in (0,1)$  | Damping factor (typically 0.85) |
|  $\mathbf{r}$  | PageRank vector (stationary distribution of node visit probabilities) |
|  $\mathbf{1}_N$  | Column vector of ones |

<h2>The PageRank Equation</h2>

$$
\mathbf{r} \;=\; \alpha\, M\, \mathbf{r} \;+\; (1-\alpha)\,\frac{1}{N}\,\mathbf{1}_N
$$

Equivalently, as a fixed point of the **Google matrix** $G$:
$$
\mathbf{r} = G\,\mathbf{r}, 
\qquad
G \;=\; \alpha\,M \;+\; (1-\alpha)\,\frac{1}{N}\,E,
$$
where $E$ is the all-ones matrix.

<h2>Iterative Computation</h2>

Start from $\,\mathbf{r}^{(0)} = \tfrac{1}{N}\,\boldsymbol{1}_N$ and iterate
$$
\mathbf{r}^{(t+1)} \;=\; \alpha\, M\, \mathbf{r}^{(t)} \;+\; (1-\alpha)\,\frac{1}{N}\,\boldsymbol{1}_N
$$
until convergence, e.g.
$$
\left\lVert \mathbf{r}^{(t+1)} - \mathbf{r}^{(t)} \right\rVert_1 \;<\; \varepsilon .
$$

<h2>Toy Example</h2>

Let’s take 3 pages: A, B, and C.
* A links to B and C
* B links to C
* C links to A

We can represent this as a graph:

$$A \rightarrow B,C$$
$$B \rightarrow C$$
$$C \rightarrow A$$

<h3>Step 1- Build the Transition Matrix</h3>
<hr>
Each column shows the probabilities from that node:

| **To ↓ / From →** | **A** | **B** | **C** |
|:------------------:|:----:|:----:|:----:|
| **A** | 0 | 0 | 1 |
| **B** | ½ | 0 | 0 |
| **C** | ½ | 1 | 0 |


In matrix form:

$$
M =
\begin{bmatrix}
0 & 0 & 1 \\
\frac{1}{2} & 0 & 0 \\
\frac{1}{2} & 1 & 0
\end{bmatrix}
$$

<h3>Step 2 - Iterative Update</h3>
<hr>

We start with equal ranks for all nodes:

$$
\mathbf{r}^{(0)} =
\begin{bmatrix}
\frac{1}{3} \\[4pt]
\frac{1}{3} \\[4pt]
\frac{1}{3}
\end{bmatrix}
$$

At each iteration, we update the rank vector using the PageRank equation:

$$
\mathbf{r}^{(t+1)} = \alpha\, M\, \mathbf{r}^{(t)} + (1 - \alpha)\, \frac{1}{N}\, \mathbb{1}_N
$$

where  

- $\alpha$ is the damping factor (typically $0.85$),  
- $M$ is the column-stochastic transition matrix, and  
- $\mathbb{1}_N$ is the all-ones vector of length $N$.

For this example with $N = 3$ and $\alpha = 0.85$, the equation becomes:

$$
\mathbf{r}^{(t+1)} = 0.85\, M\, \mathbf{r}^{(t)} + 0.15
\begin{bmatrix}
\frac{1}{3} \\[4pt]
\frac{1}{3} \\[4pt]
\frac{1}{3}
\end{bmatrix}
$$

We repeat this process until the change between iterations is small:

$$
\left\lVert \mathbf{r}^{(t+1)} - \mathbf{r}^{(t)} \right\rVert_1 < \varepsilon
$$

At convergence, $\mathbf{r}^{(t)}$ gives the **steady-state PageRank scores** for all nodes.

After several iterations, the ranks converge approximately to:

$$
\mathbf{r}^{(T)} =
\begin{bmatrix}
0.387 \\[4pt]
0.214 \\[4pt]
0.399
\end{bmatrix}
$$

This tells us:
* C is the most important
* A is the 2nd most important
* B is the least important

<h2>Implementing PageRank</h2>

Here's a simple implementation that only uses Numpy

In [ ]:
import numpy as np 

def pagerank_simple(graph, damping_factor=0.85, max_iterations=100, tol=1e-6):    
    nodes = list(graph.keys())
    n = len(nodes)
    ranks = {node: 1/n for node in nodes} # initial rank for each node
    prob = np.array([1/n for node in nodes])
    new_ranks = ranks.copy()
    
    # Create a mapping from node to index for easier computation
    node_to_index = {node: i for i, node in enumerate(nodes)}
    index_to_node = {i: node for node, i in node_to_index.items()}
    
    # Build the adjacency matrix
    adjacency_matrix = np.zeros((n, n))
    for node, neighbors in graph.items():
        if neighbors: # avoid division by zero
            for neighbor in neighbors:
                adjacency_matrix[node_to_index[neighbor], node_to_index[node]] = 1 / len(neighbors)
    
    # Iterative computation
    k = ((1 - damping_factor) / n)*np.ones(n)
    Mprime = damping_factor * adjacency_matrix
    prob = np.linalg.inv(Mprime-np.eye(n))@-k.T
    ranks = dict(zip(ranks.keys(), prob))

    return ranks

In [ ]:
graph = {
        'A': ['B', 'C'],
        'B': ['C'],
        'C': ['A']        
    }
ranks = pagerank_simple(graph)
print("PageRank scores from Simple Implementation:")
for node, score in sorted(ranks.items()):
    print(f"  {node}: {score:.6f}")


We can also use the NetworkX PageRank implementation to compute the ranks for us:

In [ ]:
#pip install networkx

In [ ]:
import networkx as nx

# Create directed graph
G = nx.DiGraph(graph)

# Compute PageRank
ranks = nx.pagerank(G, alpha=0.85)

print("PageRank scores from NetworkX:")
for node, score in sorted(ranks.items()):
    print(f"  {node}: {score:.6f}")

<h2>Example with Amazon e-commerce data</h2>

In [ ]:

# Load directed graph
path = "amazon0302.txt"
G = nx.read_edgelist(path, comments="#", create_using=nx.DiGraph(), nodetype=int)

print(f"Nodes: {G.number_of_nodes():,}")
print(f"Edges: {G.number_of_edges():,}")

# Compute PageRank
pr = nx.pagerank(G, alpha=0.85)

# Show top-10 nodes
top10 = sorted(pr.items(), key=lambda x: x[1], reverse=True)[:10]
for node, score in top10:
    print(f"Product {node:>6} → PageRank={score:.6f}")



In [ ]:
#!pip install pyvis
#!pip install jinja2

In [ ]:
import os, webbrowser
import numpy as np
from pyvis.network import Network

N = 400  # keep this reasonable for the browser
top_nodes = sorted(pr, key=pr.get, reverse=True)[:N]
subG = G.subgraph(top_nodes).copy()

# Setup
net = Network(height="800px", width="100%", directed=True,
              bgcolor="#1e1e1e", font_color="white", notebook=False)

# add nodes
vals = np.array([pr[n] for n in subG.nodes()])
vmin, vmax = vals.min(), vals.max()
def size_for(v):
    if vmax == vmin: return 10.0
    z = (v - vmin) / (vmax - vmin)
    return 6.0 + 34.0 * z

# simple buckets/colors
q99, q95, q80 = np.quantile(vals, [0.99, 0.95, 0.80])
def color_for(v):
    if v >= q99: return "#e74c3c"
    if v >= q95: return "#e67e22"
    if v >= q80: return "#f1c40f"
    return "#4aa3f0"

for n in subG.nodes():
    v = pr[n]
    net.add_node(n,
                 label=str(n),
                 title=f"Node {n}<br>PageRank={v:.6g}",
                 size=size_for(v),
                 color=color_for(v))

# add edges
for u, v in subG.edges():
    net.add_edge(u, v)

# Write HTML and open 
outfile = "amazon_network.html"
net.write_html(outfile)  
webbrowser.open("file://" + os.path.abspath(outfile))
print(f"Wrote {outfile} and opened in your browser.")


In [ ]:
# Convert to arrays
nodes = np.fromiter(pr.keys(), dtype=int, count=len(pr))
scores = np.fromiter(pr.values(), dtype=float, count=len(pr))

In [ ]:
k = 20
alpha=0.85
order = np.argsort(-scores)[:k]  # top-k by PR
print("\n=== Top nodes by PageRank ===")
print(f"(alpha={alpha})\n")
print(f"{'Rank':>4}  {'Node':>10}  {'PR':>12}  {'in_deg':>8}  {'out_deg':>8}  {'total_deg':>9}")
for r, idx in enumerate(order, start=1):
    n = nodes[idx]
    score = scores[idx]
    indeg = G.in_degree(n)
    outdeg = G.out_degree(n)
    print(f"{r:>4}  {n:>10}  {score:>12.6g}  {indeg:>8}  {outdeg:>8}  {indeg+outdeg:>9}")

From the table above, it appears there is a correlation with the number of in-degree edges and the final pagerank score.

<h2>Repurposing PageRank to Flag Anomalies</h2>

The PageRank algorithm has been repurposed quite frequently to detect anomalies. One of the readings in this week's homework assignment involves applying PageRank to nursing home residents movements to detect movements that may be a cause for concern. Here are a few common ways you can apply PageRank to find anomalies.

<h3>Outliers in the Ranks</h3>

In our dataset we have $N=262,111$ nodes. If each node were equally important, this implies each node's expected PageRank would be $1/N$ or $1/262,111 \approx 3.8\times10^{-6}$

Nodes whose PageRank score is extremely high/low relative to the global distribution are flagged. Since our PageRank results are highly skewed, we opt to flag the top and bottom 0.5% of scores.

In [ ]:
scores = np.fromiter(pr.values(), float)

# get absolute cutoff thresholds
top_cutoff    = np.percentile(scores, 99.5)   # top 0.5%
bottom_cutoff = np.percentile(scores, 0.5)    # bottom 0.5%

top_nodes = [n for n,s in pr.items() if s >= top_cutoff]
bottom_nodes = [n for n,s in pr.items() if s <= bottom_cutoff]

print("Expected avg PR:", 1 / G.number_of_nodes())
print("Top 0.5% cutoff:", top_cutoff)
print("Bottom 0.5% cutoff:", bottom_cutoff)
print("Num top:", len(top_nodes))
print("Num bottom:", len(bottom_nodes))


<h3>PageRank vs. Degree "mismatch" (residual anomalies)</h3>

If we plot the PageRank scores with In-Degree, we usually observe a significant positive correlation. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

nodes  = list(pr.keys())
pr_vals = np.array([pr[n] for n in nodes])
in_deg  = np.array([G.in_degree(n) for n in nodes], dtype=float)

# ---- (1) Raw scatter ----
plt.figure(figsize=(7,5))
plt.scatter(in_deg, pr_vals, s=8, alpha=0.5)
plt.xlabel("In-degree")
plt.ylabel("PageRank score")
plt.title("PageRank vs In-degree")
plt.grid(True, ls="--", alpha=0.3)
plt.tight_layout()
#plt.show()

a, b = np.polyfit(in_deg, pr_vals, 1)
x_line = np.linspace(in_deg.min(), in_deg.max(), 200)
plt.plot(x_line, a*x_line + b, color="red", lw=2)

plt.tight_layout()
plt.show()


The data is "fanned-out" at higher degrees and it justifies taking logarithms of both variables. However we will ignore this for now to illustrate the methodology.

In [ ]:
in_degs = np.array([G.in_degree(n) for n in nodes], dtype=float)
corr = np.corrcoef(in_degs, scores)[0,1]
print(f"\nCorrelation(PR, in-degree) ≈ {corr:.3f}")

Nodes with a much higher PageRank than their in-degree predicts (or vice-versa) can be suspicious (e.g., boosted by a few very influential neighbors). To do this we regress PageRank on in_degree and inspect residuals. 

In [ ]:
# This code will take 5-10 minutes to run
import numpy as np

nodes = list(G.nodes())
indeg = np.array([G.in_degree(n) for n in nodes], float)
X = indeg
y = np.array([pr[n] for n in nodes])

# simple linear fit
a, b = np.polyfit(X, y, 1)
resid = y - (a*X + b)

# anomalies: PR too high/low vs. degree
hi_resid = [n for n,r in zip(nodes,resid) if r > 4*resid.std()]  # fewer than expected in-degrees but neighbors are very influential
lo_resid = [n for n,r in zip(nodes,resid) if r < -4*resid.std()] # several in-degrees but very few are important


In [ ]:
print(f'Nodes with the highest residuals:{hi_resid[:10]}')